## Collaborative Filtering

In [1]:
# import pandas
import pandas as pd
from sklearn.neighbors import NearestNeighbors

We will create an recommender engine based on Item Based Collaborative Filtering (IBCF) which searches for the most similar books based on the user ratings. We can download the data from [here](https://drive.google.com/file/d/1WvTmAfO09TCX7xp7uu06__ziic7JnrL5/view?usp=sharing).

In [15]:
book_ratings = pd.read_csv('data/BX-Book-Ratings.csv',sep=";", encoding="latin")
books = pd.read_csv('data/BX-Books.csv',sep=";", encoding="latin", error_bad_lines=False)

C:\Users\thowe\AppData\Local\Temp\ipykernel_45864\202726426.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  books = pd.read_csv('data/BX-Books.csv',sep=";", encoding="latin", error_bad_lines=False)
b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957

* Explore both datasets

In [3]:
print(book_ratings.shape)
book_ratings['Book-Rating'].count()


(1149780, 3)


1149780

In [4]:
print(book_ratings['User-ID'].head())

0    276725
1    276726
2    276727
3    276729
4    276729
Name: User-ID, dtype: int64


In [5]:
print(books.shape)
books.head()

(271360, 8)


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [6]:
books = books.set_index("ISBN")


In [7]:
book_df = book_ratings.join(books,how="left",on="ISBN")
book_df.set_index('ISBN')

,User-ID,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
ISBN,,,,,,,,,
034545104X,276725,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
0155061224,276726,5,Rites of Passage,Judith Rae,2001,Heinle,http://images.amazon.com/images/P/0155061224.0...,http://images.amazon.com/images/P/0155061224.0...,http://images.amazon.com/images/P/0155061224.0...
0446520802,276727,0,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...
052165615X,276729,3,Help!: Level 1,Philip Prowse,1999,Cambridge University Press,http://images.amazon.com/images/P/052165615X.0...,http://images.amazon.com/images/P/052165615X.0...,http://images.amazon.com/images/P/052165615X.0...
0521795028,276729,6,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,2001,Cambridge University Press,http://images.amazon.com/images/P/0521795028.0...,http://images.amazon.com/images/P/0521795028.0...,http://images.amazon.com/images/P/0521795028.0...
...,...,...,...,...,...,...,...,...,...
1563526298,276704,9,Get Clark Smart : The Ultimate Guide for the S...,Clark Howard,2000,Longstreet Press,http://images.amazon.com/images/P/1563526298.0...,http://images.amazon.com/images/P/1563526298.0...,http://images.amazon.com/images/P/1563526298.0...
0679447156,276706,0,Eight Weeks to Optimum Health: A Proven Progra...,Andrew Weil,1997,Alfred A. Knopf,http://images.amazon.com/images/P/0679447156.0...,http://images.amazon.com/images/P/0679447156.0...,http://images.amazon.com/images/P/0679447156.0...
0515107662,276709,10,The Sherbrooke Bride (Bride Trilogy (Paperback)),Catherine Coulter,1996,Jove Books,http://images.amazon.com/images/P/0515107662.0...,http://images.amazon.com/images/P/0515107662.0...,http://images.amazon.com/images/P/0515107662.0...


* create dataframe with name 'df_book_features' from book_ratings that have `ISBN` as index, `User-ID` as columns and values are `Book-Rating`.
    - The data are quite big so it's OK to use a sample only in case your PC has limited RAM.


In [8]:
del books
del book_ratings

In [89]:
import numpy as np
df_book_features = book_df.sample(frac=0.025,random_state=1738).pivot(index=['ISBN'],columns=['User-ID'],values=['Book-Rating'])

from sklearn.impute import  SimpleImputer
imputer = SimpleImputer(missing_values=np.nan,strategy="mean")

df_book_features2 = pd.DataFrame(imputer.fit_transform(X=df_book_features.T).T)
# df_book_features = df_book_features.fillna(-100)

In [90]:
df_book_features2.columns = df_book_features.columns
df_book_features2 = df_book_features2.set_index(df_book_features.index)


In [97]:
df_book_features = df_book_features2
df_book_features

Book-Rating                                                   \
User-ID             38     99     182    183    243    244    254    272      
ISBN                                                                          
0.473.087979           8.0    8.0    8.0    8.0    8.0    8.0    8.0    8.0   
000000000              0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
0000000000            10.0   10.0   10.0   10.0   10.0   10.0   10.0   10.0   
000001246971           8.0    8.0    8.0    8.0    8.0    8.0    8.0    8.0   
0001046624             0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
...                    ...    ...    ...    ...    ...    ...    ...    ...   
Q912838027             0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
SBN425037452           0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
X000000000            10.0   10.0   10.0   10.0   10.0   10.0   10.0   10.0   
\8887517452\""        10.0   10.0   10.0   10.0   10.0   10.0   10.0   10.0   
cn108465               0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

                              ...                                            \
User-ID        392    440     ... 278563 278582 278637 278644 278659 278691   
ISBN                          ...                                             
0.473.087979      8.0    8.0  ...    8.0    8.0    8.0    8.0    8.0    8.0   
000000000         0.0    0.0  ...    0.0    0.0    0.0    0.0    0.0    0.0   
0000000000       10.0   10.0  ...   10.0   10.0   10.0   10.0   10.0   10.0   
000001246971      8.0    8.0  ...    8.0    8.0    8.0    8.0    8.0    8.0   
0001046624        0.0    0.0  ...    0.0    0.0    0.0    0.0    0.0    0.0   
...               ...    ...  ...    ...    ...    ...    ...    ...    ...   
Q912838027        0.0    0.0  ...    0.0    0.0    0.0    0.0    0.0    0.0   
SBN425037452      0.0    0.0  ...    0.0    0.0    0.0    0.0    0.0    0.0   
X000000000       10.0   10.0  ...   10.0   10.0   10.0   10.0   10.0   10.0   
\8887517452\""   10.0   10.0  ...   10.0   10.0   10.0   10.0   10.0   10.0   
cn108465          0.0    0.0  ...    0.0    0.0    0.0    0.0    0.0    0.0   

                                            
User-ID        278771 278789 278843 278851  
ISBN                                        
0.473.087979      8.0    8.0    8.0    8.0  
000000000         0.0    0.0    0.0    0.0  
0000000000       10.0   10.0   10.0   10.0  
000001246971      8.0    8.0    8.0    8.0  
0001046624        0.0    0.0    0.0    0.0  
...               ...    ...    ...    ...  
Q912838027        0.0    0.0    0.0    0.0  
SBN425037452      0.0    0.0    0.0    0.0  
X000000000       10.0   10.0   10.0   10.0  
\8887517452\""   10.0   10.0   10.0   10.0  
cn108465          0.0    0.0    0.0    0.0  

[22579 rows x 11365 columns]

* create the instance of the NearestNeighbors class

In [98]:
from sklearn.neighbors import NearestNeighbors

nn = NearestNeighbors(n_jobs=-1,n_neighbors=5)

In [99]:
# df_book_features.head()

* fit the NearestNeighbors using'df_book_features'

In [100]:
model = nn.fit(df_book_features)

* create function that returns top 5 most similar movies (according to KNN model) for selected ISBN
    * the input will be Book-Title from the DataFrame books 
    * the output will be the Book-Titles of the top 5 most similar books.
    * for every book in the top 5 most similar books, print also the distance from the selected book (ISBN we chose as input to the function)

In [110]:
import numpy as np
def get_similar(book_title:str) -> pd.DataFrame:

    the_book = books.loc[books['Book-Title'] == book_title]
    the_isbn = str(the_book.iloc[0,:]['ISBN'])

    print(the_isbn)
    the_row = np.array(df_book_features.loc[str(the_isbn),:])
    the_row = the_row.reshape(1,-1)

    rez = model.kneighbors(the_row,5)
    print(rez)
    ilist = list(df_book_features.index)
    out_df = pd.DataFrame(columns=['Book-Title','Distance'])
    out_df['Distance'] = rez[0][0]
    title_list = []
    for i in rez[1][0]:
        isbn = ilist[i]
        title_list = title_list + (books.loc[books['ISBN'] == isbn])['Book-Title']

    out_df['Book-Title'] = title_list
    return out_df


* Apply the function to book of your choice

In [111]:
get_similar("The Notebook")

0446605239
(array([[ 0.       , 15.3622915, 15.3622915, 15.3622915, 15.3622915]]), array([[8857,  925,  792, 3038,  104]], dtype=int64))


ValueError: Length of values (0) does not match length of index (1)

In [53]:
df_book_features.loc['0446605239',:]

             User-ID
Book-Rating  38        -100.0
             99        -100.0
             182       -100.0
             183       -100.0
             243       -100.0
                        ...  
             278691    -100.0
             278771    -100.0
             278789    -100.0
             278843    -100.0
             278851    -100.0
Name: 0446605239, Length: 11365, dtype: float64